In [1]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

import ccxt as cx
from typing import Any, Optional, List
import ccxt.pro as cxp
from ccxt.base.decimal_to_precision import ROUND_UP
import asyncio
import nest_asyncio
nest_asyncio.apply()
from ccxt.base.exchange import Exchange

import configparser

import qubx
%qubxd dev

%load_ext autoreload
%autoreload 2

from qubx import lookup, logger
from qubx.core.series import TimeSeries, Bar, Trade, Quote, OHLCV
from qubx.utils.pandas import scols, ohlc_resample
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent
from qubx.core.basics import Instrument, Position, Signal, TransactionCostsCalculator, CtrlChannel, AsyncioThreadRunner
from qubx.impl.ccxt_connector import CCXTConnector, BinanceQV
from qubx.utils.pandas import ohlc_resample

def get_account_auth(acc_name):
    parser = configparser.ConfigParser()
    parser.optionxform=str
    parser.read('.env')
    return dict(parser[acc_name])

 >  [dev] installing cython rebuilding hook

                   ╻
   ┏┓      ╻       ┃  ┏┓       Quantitative Backtesting Environment 
   ┃┃  ┓┏  ┣┓  ┏┓  ┃  ┏┛       
   ┗┻  ┗┻  ┗┛  ┗   ┃  ┗━       (c) 2024,  ver. Dev
                   ╹       



# Common

In [ ]:
lookup.instruments['KRAKEN.F:BTC/USD']

In [34]:
instr = lookup.instruments.find('BINANCE.UM', 'BTC', 'USDT')
instr

Instrument(symbol='BTCUSDT', market_type='CRYPTO', exchange='BINANCE.UM', base='BTC', quote='USDT', margin_symbol='USDT', min_tick=0.1, min_size_step=0.001, min_size=0.001, futures_info=FuturesInfo(contract_type='PERPETUAL', delivery_date=datetime.datetime(2100, 12, 25, 10, 0), onboard_date=datetime.datetime(2019, 9, 25, 11, 0), contract_size=1.0, maint_margin=2.5, required_margin=5.0, liquidation_fee=0.0125), _aux_instrument=None)

In [39]:
instr = lookup.instruments.find('KRAKEN', 'BTC', 'ETH')
instr

Instrument(symbol='ETH/BTC', market_type='spot', exchange='KRAKEN', base='ETH', quote='BTC', margin_symbol=None, min_tick=1e-05, min_size_step=1e-05, min_size=1e-08, futures_info=None, _aux_instrument=None)

In [38]:
lookup.find_aux_instrument_for(instr, 'USD')

Instrument(symbol='BTC/USD', market_type='spot', exchange='KRAKEN', base='BTC', quote='USD', margin_symbol=None, min_tick=0.1, min_size_step=0.1, min_size=1e-08, futures_info=None, _aux_instrument=None)

In [46]:
bu = cxp.binanceusdm()
bs = cxp.binance()
ks = cxp.kraken()
kf = cxp.krakenfutures()

# Get OHLC [TODO]

In [47]:
bs_markets = await bs.load_markets()
bu_markets = await bu.load_markets()
kf_markets = await kf.load_markets()
ks_markets = await ks.load_markets()

In [53]:
bu.parse_timeframe('1m')

60

In [ ]:
interval = bu.parse_timeframe('1m') * 1000
orderbook_max_levels_limit = 100
since = bu.round_timeframe('1m', bu.milliseconds(), ROUND_UP) - orderbook_max_levels_limit * interval
await bu.fetch_ohlcv('ETH/USDT', '1m', since=since, limit=orderbook_max_levels_limit)

In [ ]:
# await bnc1.fetch_ohlcv('ETH/USDT', '1m', since=since, limit=limit)

In [57]:
cxc = CCXT_connector('binance')

# <font color='#ff0000'>Listen to data></font>

In [12]:
import asyncio
# from multiprocessing import Queue, Process, Event, Lock
from threading import Thread, Event, Lock
from queue import Queue
# from multiprocessing import Queue, Process, Event, Lock
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

In [23]:
class TestExch:
    def __init__(self, exch: Exchange) -> None:
        self.exchange = exch
        self.loop = asyncio.get_running_loop()

    async def listen_to_trades(self, channel: CtrlChannel, symbols: List[str]):
        print("Start watch_trades_for_symbols ...")
        data = asyncio.run(self.fetch_ohlcs(symbols[0], '1m', 10))
        print('INIT:', data)

        while channel.control.is_set():
            try:
                trades = await self.exchange.watch_trades_for_symbols(symbols)
                for trade in trades:
                    t_ns = trade['timestamp'] * 1_000_000 # this is trade time 
                    s, info, price, amnt = trade['symbol'], trade['info'], trade['price'], trade['amount']
                    m = info['m']
                    tas = Trade(t_ns, price, amnt, int(not m), int(trade['id']))
                    channel.queue.put((s, tas))
            except Exception as e:
                print(type(e).__name__, str(e), flush=True)
                await self.exchange.close()
                raise e
        print("Stop watch_trades_for_symbols")

    async def listen_to_orderbook(self, channel: CtrlChannel, symbols: List[str], orderbook_max_levels_limit=5):
        while channel.control.is_set():
            try:
                orderbook = await self.exchange.watch_order_book_for_symbols(symbols, limit=orderbook_max_levels_limit)
                t_ns = orderbook['timestamp'] * 1_000_000
                s, b, a = orderbook['symbol'], orderbook['bids'], orderbook['asks']
                q = Quote(t_ns, b[0][0], a[0][0], b[0][1], a[0][1])
                channel.queue.put((s, q))
            except Exception as e:
                print(type(e).__name__, str(e), flush=True)
                await self.exchange.close()
                raise e

    async def fetch_ohlcs(self, symbol: str, timeframe: str, nbarsback: int):
        start = ((pd.Timestamp('now', tz='UTC') - nbarsback * pd.Timedelta(timeframe)).asm8.item()//1000000) if nbarsback > 1 else None 
        print("> START fetching OHLCV...")
        return await self.exchange.fetch_ohlcv(symbol, timeframe, since=start, limit=nbarsback + 1)

    async def receiver(self, channel: CtrlChannel):
        sers = {}
        print(">>> START LISTENING", flush=True)
        T, Q = 0, 0
        while channel.control.is_set():
            s, data = channel.queue.get()
            if isinstance(data, Quote):
                if not (Q % 100):
                    print(s, data, flush=True)
                    Q = 0
                Q += 1
            if isinstance(data, Trade):
                if s not in sers:
                    sers[s] = TimeSeries(s, '1Min')
                ts = sers[s]
                ts.update(data.time, data.price)
                if not (T % 200):
                    print(s, data, flush=True)
                    T = 0
                    print("--- Send request for OHLCV ...")
                    t1 = self.loop.create_task(self.fetch_ohlcs(s, '1m', 10))
                    while not t1.done():
                        await asyncio.sleep(0.01)
                    print("DATA: ", t1.result())
                    # print("DATA: ", data)
                T += 1
            if isinstance(data, Bar):
                # print(f"{s} {pd.Timestamp(data.time, unit='ns')}: {data}", flush=True)
                if s not in sers:
                    sers[s] = {}
                sers[s][data.time] = data

        print(f">>> STOP LISTENING: {str(sers)}", flush=True)
        return sers 

In [24]:
ctx = TestExch(BinanceQV())

In [25]:
ch = CtrlChannel('A')
rn0 = AsyncioThreadRunner(ch)

In [26]:
loop = asyncio.get_running_loop()
t0 = loop.create_task(ctx.listen_to_trades(ch, ['BTCUSDT']))
t1 = loop.create_task(ctx.fetch_ohlcs('BTCUSDT', '1m', 10))
rn0.add( ctx.receiver)
rn0.start()

# c2 = await loop.run_in_executor(pool2, ctx.listen_to_trades, ch, ['BTCUSDT'])
# await asyncio.gather(c1, c2)

>>> START LISTENING


Start watch_trades_for_symbols ...
> START fetching OHLCV...
> START fetching OHLCV...
INIT: [[1711971720000, 69443.7, 69443.71, 69440.0, 69440.0, 5.89833, 409586.5446374, 271984.9707982], [1711971780000, 69440.01, 69500.0, 69440.0, 69490.01, 20.01146, 1390074.529704, 1036310.850841], [1711971840000, 69490.01, 69540.0, 69490.0, 69510.96, 14.81056, 1029563.0785195, 493610.8801056], [1711971900000, 69510.96, 69570.83, 69500.0, 69554.09, 19.08319, 1327018.2596471, 962626.2588972], [1711971960000, 69554.08, 69554.09, 69519.99, 69520.0, 9.45391, 657416.3889968, 251547.7517552], [1711972020000, 69519.99, 69520.0, 69498.41, 69498.41, 12.73967, 885420.2214033, 387573.7884844], [1711972080000, 69498.42, 69498.42, 69480.32, 69480.33, 5.39792, 375095.2328076, 60109.1915999], [1711972140000, 69480.32, 69480.33, 69455.79, 69456.19, 15.11164, 1049695.3534969, 121858.6134701], [1711972200000, 69456.19, 69480.0, 69456.18, 69480.0, 15.69416, 1090305.8098581, 979757.3004303], [1711972260000, 69479.99, 6

In [27]:
ch.stop()

>>> STOP LISTENING: {'BTC/USDT': 2024-04-01 11:51:00    69479.99
Name: BTC/USDT, dtype: float64}


In [58]:
# rn0.stop()
# rn1.stop()

BTC/USDT [2024-04-01T11:24:49.455000000]	69610.01000 (0.0) <take> 3524799974
--- Send request for OHLCV ...
> START fetching OHLCV...
DATA:  [[1711970100000, 69724.01, 69724.01, 69700.01, 69709.76, 14.86888, 1036521.5927524, 617810.9342986], [1711970160000, 69709.76, 69723.88, 69709.75, 69720.78, 11.73767, 818351.0562077, 269665.2146643], [1711970220000, 69720.78, 69760.0, 69700.23, 69700.24, 33.50795, 2336411.6094154, 1296220.2618832], [1711970280000, 69700.23, 69710.28, 69692.02, 69693.7, 17.94041, 1250488.2348796, 511502.5382097], [1711970340000, 69693.7, 69693.7, 69660.36, 69660.49, 7.59259, 529021.1224605, 168510.3359819], [1711970400000, 69660.48, 69672.0, 69620.78, 69620.78, 21.80716, 1519151.1035091, 490904.8042106], [1711970460000, 69620.79, 69630.97, 69617.65, 69630.96, 13.73049, 955991.950191, 468183.1579287], [1711970520000, 69630.96, 69648.0, 69620.0, 69632.69, 22.78384, 1586433.187442, 1058745.1291371], [1711970580000, 69632.68, 69650.18, 69632.68, 69650.18, 14.06183, 979

In [4]:
def prefetch_ohlcs(exchange: Exchange, symbol: str, timeframe: str, nbarsback: int):
    assert nbarsback > 1
    start = ((pd.Timestamp('now', tz='UTC') - nbarsback * pd.Timedelta(timeframe)).asm8.item()//1000000) if nbarsback > 1 else None 
    ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since=start, limit=nbarsback + 1)
    return ohlcv

async def listen_to_ohlcv(channel: CtrlChannel, exchange: Exchange, 
                          symbol: List[str], timeframe: str, nbarsback: int):
    # - check if we need to load initial 'snapshot'
    if nbarsback > 1:
        ohlcv = await prefetch_ohlcs(exchange, symbol, timeframe, nbarsback)
        for oh in ohlcv:
            b = Bar(oh[0] * 1_000_000, oh[1], oh[2], oh[3], oh[4], oh[6], oh[7])
            channel.queue.put((symbol, b))

    while channel.control.is_set():
        try:
            ohlcv = await exchange.watch_ohlcv(symbol, timeframe)
            for oh in ohlcv:
                b = Bar(oh[0] * 1000000, oh[1], oh[2], oh[3], oh[4], oh[6], oh[7])
                channel.queue.put((symbol, b))
        except Exception as e:
            print(type(e).__name__, str(e), flush=True)
            await exchange.close()
            raise e

In [5]:
symbs = ['1INCHUSDT', 'AGLDUSDT', 'APEUSDT', 'ARBUSDT', 'AUCTIONUSDT', 'BANDUSDT', 'BATUSDT', 'C98USDT', 'COMPUSDT', 'CYBERUSDT',
        'DYDXUSDT', 'ENSUSDT', 'FXSUSDT', 'HFTUSDT', 'ICXUSDT', 'IOSTUSDT', 'IOTAUSDT', 'IOTXUSDT', 'LDOUSDT', 'LQTYUSDT', 'LTCUSDT',
        'MASKUSDT', 'MBOXUSDT', 'MINAUSDT', 'MTLUSDT', 'OGNUSDT', 'OPUSDT', 'PEOPLEUSDT', 'POLSUSDT', 'PYRUSDT', 'QNTUSDT', 'RADUSDT',
        'RDNTUSDT', 'RIFUSDT', 'RLCUSDT', 'SEIUSDT', 'SPELLUSDT', 'SSVUSDT', 'STGUSDT', 'STXUSDT', 'SUIUSDT', 'UMAUSDT', 'WAVESUSDT']

In [6]:
# chan = CtrlChannel('ticks')
# reader = RunThread(chan)
# reader.add(receiver)
# reader.start()

START LISTENING


In [7]:
# bu = cxp.binanceusdm()
# bu = cxp.binance()
# bu = BinanceQV()
# bu = getattr(cxp, 'binanceqv')()
# t1 = RunThread(chan)
# for s in symbs:
#     t1.add(listen_to_ohlcv, bu, s, '15m', 10)

# t1.add(listen_to_ohlcv, bu, 'BTCUSDT', '5m', 10)
# t1.add(listen_to_ohlcv, bu, 'ETHUSDT', '5m', 10)
# t1.add(listen_to_ohlcv, bu, 'LTCUSDT', '5m', 10)
# t1.add(listen_to_ohlcv, bu, 'SOLUSDT', '5m', 10)
# t1.add(listen_to_trades, bu, ['BTCUSDT', 'ETHUSDT', 'SOLUSDT'])
# t1.add(listen_to_orderbook, bu, ['BTCUSDT', 'ETHUSDT', 'SOLUSDT'])
# t1.start()

In [10]:
reader.stop()
t1.stop()

In [ ]:
print(len(reader.result[0]), len(symbs))
reader.result[0]

# StrategyCtx

In [10]:
%load_ext autoreload
%autoreload 2

from qubx.core.basics import Instrument, Position, Signal
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
class ZeroStrat(IStrategy):
    param1: int = 0

    def on_start(self, ctx: StrategyContext):
        logger.info(f"> Started with param1 = {self.param1}")

    def on_event(self, ctx: StrategyContext, event: TriggerEvent) -> Optional[List[Signal]]:
        logger.info(f"{event.time} -> {event}")
        xs = scols(*[ctx.ohlc(s, '1Min').pd()['close'].rename(s.symbol) for s in ctx.instruments])
        print(xs)
        data = ctx.data_provider.get_historical_ohlcs('BTCUSDT', '1Min', 10)
        print(data)
        return None

    def on_stop(self, ctx: StrategyContext):
        logger.info(f"> stopped")

In [12]:
cc = CCXTConnector('binance', **get_account_auth('account1'))
cc.time()
# await cc.exchange.fetch_positions(['BTCUSDT'])

numpy.datetime64('2024-04-02T09:20:44.365967000')

In [211]:
async def _get_orders(self, symbol: str, days_before=365):
    orders = {}
    t0 = ((pd.Timestamp('now', tz='UTC') - days_before * pd.Timedelta('1d')).asm8.item()//1000000)
    ord_data =  await self.exchange.fetch_orders(symbol, since=t0)

    for o in ord_data:
        oi = o['info']
        orders[oi['orderId']] = dict(
            time=pd.Timestamp(o['timestamp'], unit='ms'),
            quantity=float(oi['executedQty']), 
            exec=float(oi['price']),
            status=oi['status'],
            side=oi['side'],
            cost=float(o['cost']),
        )
    return dict(sorted(orders.items(), key=lambda x: x[1]['time'], reverse=True))

In [192]:
balance = await cc.exchange.fetch_balance()
acc_type = balance['info']['accountType']

In [210]:
for ai in balance['info']['balances']:
    vol = float(ai['free']) + float(ai['locked'])
    if vol > 0:
        print(ai)

{'asset': 'BNB', 'free': '0.19946439', 'locked': '0.00000000'}
{'asset': 'USDT', 'free': '69.02317702', 'locked': '0.00000000'}
{'asset': 'ACA', 'free': '150.00000000', 'locked': '0.00000000'}
{'asset': 'ETHW', 'free': '0.02432953', 'locked': '0.00000000'}


In [199]:
orders = await _get_orders(cc, 'ACAUSDT', 365)

In [ ]:
orders

In [209]:
pos = 150
last_pos_orders = []
for o in orders.values():
    if o['status'] == 'FILLED':
        pos -= o['quantity'] 
        last_pos_orders.append(o)
        if pos == 0:
            break
last_pos_orders

[{'time': Timestamp('2024-04-01 14:53:57.405000'),
  'quantity': 50.0,
  'exec': 0.165,
  'status': 'FILLED',
  'side': 'BUY',
  'cost': 8.25},
 {'time': Timestamp('2024-04-01 14:53:03.356000'),
  'quantity': 50.0,
  'exec': 0.1716,
  'status': 'FILLED',
  'side': 'BUY',
  'cost': 8.58},
 {'time': Timestamp('2024-04-01 13:38:03.891000'),
  'quantity': 50.0,
  'exec': 0.0,
  'status': 'FILLED',
  'side': 'BUY',
  'cost': 8.813596}]

In [207]:
len(orders.values())

4

In [182]:
p = Position('')
for o in reversed(last_pos_orders):
    pass

[{'time': Timestamp('2024-04-01 14:53:03.356000'),
  'quantity': 50.0,
  'exec': 0.1716,
  'status': 'FILLED',
  'side': 'BUY',
  'cost': 8.58},
 {'time': Timestamp('2024-04-01 13:38:03.891000'),
  'quantity': 50.0,
  'exec': 0.0,
  'status': 'FILLED',
  'side': 'BUY',
  'cost': 8.813596}]

In [ ]:
p.update_position()

In [ ]:
orders

In [145]:
for o in ord_data:
    print(o['info'])

{'symbol': 'ACAUSDT', 'orderId': '148022194', 'orderListId': '-1', 'clientOrderId': 'electron_ce63fa14b5da4f5a99ac60679e9', 'price': '0.00000000', 'origQty': '50.00000000', 'executedQty': '50.00000000', 'cummulativeQuoteQty': '8.81359600', 'status': 'FILLED', 'timeInForce': 'GTC', 'type': 'MARKET', 'side': 'BUY', 'stopPrice': '0.00000000', 'icebergQty': '0.00000000', 'time': '1711978683891', 'updateTime': '1711978683891', 'isWorking': True, 'workingTime': '1711978683891', 'origQuoteOrderQty': '0.00000000', 'selfTradePreventionMode': 'EXPIRE_MAKER'}
{'symbol': 'ACAUSDT', 'orderId': '148035855', 'orderListId': '-1', 'clientOrderId': 'electron_d29b06e17def4150ab57ee416e9', 'price': '0.17000000', 'origQty': '50.00000000', 'executedQty': '0.00000000', 'cummulativeQuoteQty': '0.00000000', 'status': 'NEW', 'timeInForce': 'GTC', 'type': 'LIMIT', 'side': 'BUY', 'stopPrice': '0.00000000', 'icebergQty': '0.00000000', 'time': '1711982582392', 'updateTime': '1711982582392', 'isWorking': True, 'work

In [146]:
pd.Timestamp(o['timestamp'], unit='ms')
o['filled'], o['price'], o['info']['status'], o['side'], o['id']

(0.0, 0.17, 'NEW', 'buy', '148035855')

In [ ]:
o

In [ ]:
# await _init_account(cc)

In [30]:
free_bal['USDT']

85.85317702

In [ ]:
r = await cc.exchange.fetch_closed_orders(symbol='ACAUSDT')

In [ ]:
r

In [ ]:
r#['balances']

In [8]:
await cc.exchange.fetch_positions(['ACAUSDT'])

[]

In [18]:
# cc.get_historical_ohlcs('BTCUSDT', '1d', 20)

In [13]:
ctx = StrategyContext(
    ZeroStrat(), dict(param1 = 1), cc, cc, 
    instruments=lookup.instruments[r'BINANCE:(BTC|ETH|LTC|SUSHI)USDT'],
    capital=1000, fees_spec='vip0_usdt', base_currency='USDT',
    md_subscription=dict(type='ohlc', timeframe='1Min', nback=10),
    trigger=dict(type='bar', delay='-1s', timeframe='1Min')
)

2024-04-02 12:20:55.630 [ ℹ️ ] Set param1 -> 1


In [14]:
ctx.start()

2024-04-02 12:20:56.797 [ ℹ️ ] Subscribing to ohlc updates using {'timeframe': '1Min', 'nback': 10} for 
	['BTCUSDT', 'ETHUSDT', 'LTCUSDT', 'SUSHIUSDT'] 
2024-04-02 12:20:56.798 [ ℹ️ ] Subscribed on ohlc updates for 4 symbols: 
		['BTCUSDT', 'ETHUSDT', 'LTCUSDT', 'SUSHIUSDT']
2024-04-02 12:20:56.799 [ ℹ️ ] > Started with param1 = 1
2024-04-02 12:20:56.800 [ ℹ️ ] > Data processor started
2024-04-02 12:20:56.804 [ ℹ️ ] Start processing market data


In [8]:
await cc.exchange.fetch_balance()

{'info': {'makerCommission': '10',
  'takerCommission': '10',
  'buyerCommission': '0',
  'sellerCommission': '0',
  'commissionRates': {'maker': '0.00100000',
   'taker': '0.00100000',
   'buyer': '0.00000000',
   'seller': '0.00000000'},
  'canTrade': True,
  'canWithdraw': True,
  'canDeposit': True,
  'brokered': False,
  'requireSelfTradePrevention': False,
  'preventSor': False,
  'updateTime': '1712024700808',
  'accountType': 'SPOT',
  'balances': [{'asset': 'BTC', 'free': '0.00000000', 'locked': '0.00000000'},
   {'asset': 'LTC', 'free': '0.00000000', 'locked': '0.00000000'},
   {'asset': 'ETH', 'free': '0.00000000', 'locked': '0.00000000'},
   {'asset': 'NEO', 'free': '0.00000000', 'locked': '0.00000000'},
   {'asset': 'BNB', 'free': '0.19946439', 'locked': '0.00000000'},
   {'asset': 'QTUM', 'free': '0.00000000', 'locked': '0.00000000'},
   {'asset': 'EOS', 'free': '0.00000000', 'locked': '0.00000000'},
   {'asset': 'SNT', 'free': '0.00000000', 'locked': '0.00000000'},
   {'

In [15]:
ctx.stop()

2024-04-02 12:21:02.436 [ ℹ️ ] > stopped
2024-04-02 12:21:02.436 [ ℹ️ ] Market data processing finished


In [ ]:
# cc.get_historical_ohlcs('BTCUSDT', '1m', 10)

In [ ]:
ctx.ohlc('SUSHIUSDT', '1Min')

In [ ]:
await cc.exchange.fetch_positions(['BTCUSDT'])

In [117]:
await cc.exchange.close()

In [ ]:
ctx._cache._ohlcvs['SUSHIUSDT']

<hr/>

# Asyncs